파이썬으로 멀티스레드와 멀티프로세스 구현하는 방법을 알아보자. `concurrent.futures`를 이용해 벙렬 프로그래밍을 구현해보자.   
__파이썬 3.7.9 버전을 기준으로 작성되었다. 버전업으로 인한 상이한점은 주의하자__

# 30-2. 멀티태스킹(1) 멀티태스킹이란?

어떻게 한정된 컴퓨팅 자원을 활용하여 여러 가지 일을 효율적으로 진행할 수 있을까? 멀티태스킹은 2가지 동작 방식이 있는데 동시성과 병렬성으로 효율적으로 진행할 수 있다. [Difference Between Concurrency and Parallelism](https://techdifferences.com/difference-between-concurrency-and-parallelism.html)   

하나의 precessor가 여러 task를 동시에 수행하는 개념을 __동시성__ 이라고 한다. 실제로 특정 순간에는 한가지의 task를 수행하지만, 다른 task를 수행할 수 있는 시간에는 task를 전환해서 효율적으로 여러개를 동시에 수행하는 것 처럼 보이는 것이다.   
유사한 task를 여러 processor가 동시에 수행하는 것을 __병렬성__ 이라고 한다. precessor의 개수만큼 퍼포먼스가 좋아진다. 병렬성의 효율 극대화는 동시성이 같이 요구될때이다. 여러개의 프로세스가 1개의 task를 여러개의 subtask로 쪼개어 병렬적으로 수행할 수 있기 때문이다.   

동기(synchronous)와 비동기(asynchronous)는 동시성에서 주로 다루는 개념이다. 일을 하지 못하고 대기하는 상황을 컴퓨터에서는 bound 되었다고 표현한다. 만약 어떤 일이 바운드 상태면 계속 기다려야할지, 아니면 종료하고 다른걸 실행할지 선택해야한다. 앞 작업이 종료되기를 기다리는걸 __동기__, 다른 일을 처리하는 것을 __비동기__ 방식이라고 한다.   

컴퓨터가 일을 수행하면서 속도에 제한이 걸릴때는 I/O바운드, CPU바운드 경우가 대부분이다. I/O는 입출력 과정에서 데이터 처리에 시간이 소요될 때를 말하고, CPU는 복잡한 수식 계산이나 그래픽 작업같은 수많은 계산이 필요할 때를 말한다. [What do the terms “CPU bound” and “I/O bound” mean?](https://stackoverflow.com/questions/868568/what-do-the-terms-cpu-bound-and-i-o-bound-mean) 글을 참고하면 더 확인할 수 있다.

# 30-3. 멀티태스킹(2) 프로세스, 스레드, 프로파일링

컴퓨터를 하면서 여러 프로그램을 동시에 실행하는 멀티태스킹이 기본 사양이 됐다. 그리고 게임, 영상 통신 부분에서 멀티 스레드를 사용한다.   

게임에서 메인 스레드는 UI를 그려준다. 그래픽 부분 담당 코드는 순차적 실행이라 UI를 그리는 서버 통신 담당 소켓 부분을 방치할 수 밖에 없다. 해결하기 위해 통신 담당하는 스레드를 따로 하나를 만들어 일정 시간 단위로 체크할 수 있도록 구현하기 위해 멀티 스레드를 사용한다.   
영상통신은 영상을 받아 화면에 출력하는 코드와 영상을 생성해서 보내주는 코드 두 가지만 생각해도 여러 작업이 동시에 일어남을 알 수 있는데, 이를 구현하기 위해 다중 스레드를 사용하게 된다.

__프로세스__   
하나의 프로그램을 실행할 때 운영체제는 한 프로세스를 생성한다. 프로세스는 운영체제의 커널에서 시스템 자원 및 자료구조를 이용한다.   
프로세스는 프로그램을 구동하여 프로그램 자체와 프로그램의 상태가 메모리상에서 실행되는 작업 단위를 말한다. 예를들어 하나의 프로그램을 한 번 구동하면 하나의 프로세스가 메모리에서 실행되지만 여러 번 구동하면 여러 개의 프로세스가 실행된다. 아래 코드를 통해 `os` 모듈에서 프로세스 관련 정보를 얻을 수 있다.

In [1]:
import os

# process ID
print("process ID:", os.getpid())

# user ID
print("user ID:", os.getuid())

# group ID
print("group ID:", os.getgid())

# 현재 작업중인 디렉토리
print("current Directory:", os.getcwd())

process ID: 145
user ID: 0
group ID: 0
current Directory: /aiffel/aiffel/AI_study/Fundamentals


__스레드__   
프로그램 내에서 많은 작업이 있다. 스레드(thread)는 프로세스 내에서 실행되는 흐름의 단위이다.   
예를들어 요리 만드는 프로그램이면 요리사는 프로세스에 밥 짓기, 재료 손질하기 등 이러한 작업을 스레드에 비유할 수 있다. 같은 작업을 빠르게 처리하기 위해 여러 개의 스레드를 생성하기도한다.   
스레드의 사용은 프로그램마다 다르다. 하나의 프로그램에서 여러 스레드가 병렬적으로 처리되거나, 가벼운 프로그램은 하나의 스레드를 갖고 처리하기도한다.
![](https://d3s0tskafalll9.cloudfront.net/media/images/F-37-5.max-800x600.png)

__프로파일링(profiling)__   
프로파일링을 통해 코드에서 시스템의 어느 부분이 느린지 혹은 어디서 RAM을 많이 사용하고 있는지 확인할 수 있다. 파이썬에서는 다음 명령어로 확인할 수 있다.   
```
%timeit

import time
time.time()

import timeit
timeit.Timer()
```

In [2]:
import timeit
        
def f1():
    s = set(range(100))

    
def f2():
    l = list(range(100))

    
def f3():
    t = tuple(range(100))


def f4():
    s = str(range(100))

    
def f5():
    s = set()
    for i in range(100):
        s.add(i)

def f6():
    l = []
    for i in range(100):
        l.append(i)
    
def f7():
    s_comp = {i for i in range(100)}

    
def f8():
    l_comp = [i for i in range(100)]
    

if __name__ == "__main__":
    t1 = timeit.Timer("f1()", "from __main__ import f1")
    t2 = timeit.Timer("f2()", "from __main__ import f2")
    t3 = timeit.Timer("f3()", "from __main__ import f3")
    t4 = timeit.Timer("f4()", "from __main__ import f4")
    t5 = timeit.Timer("f5()", "from __main__ import f5")
    t6 = timeit.Timer("f6()", "from __main__ import f6")
    t7 = timeit.Timer("f7()", "from __main__ import f7")
    t8 = timeit.Timer("f8()", "from __main__ import f8")
    print("set               :", t1.timeit(), '[ms]')
    print("list              :", t2.timeit(), '[ms]')
    print("tuple             :", t3.timeit(), '[ms]')
    print("string            :", t4.timeit(), '[ms]')
    print("set_add           :", t5.timeit(), '[ms]')
    print("list_append       :", t6.timeit(), '[ms]')
    print("set_comprehension :", t5.timeit(), '[ms]')
    print("list_comprehension:", t6.timeit(), '[ms]')

set               : 1.6746660289973079 [ms]
list              : 0.7682238699999289 [ms]
tuple             : 0.7503654190004454 [ms]
string            : 0.44992789400203037 [ms]
set_add           : 5.852101393000339 [ms]
list_append       : 5.145788623998669 [ms]
set_comprehension : 5.811907856998005 [ms]
list_comprehension: 5.266198647997953 [ms]


시간을 측정하는 방법이라 프로파일링이라 하기 좀 그렇다. 엄밀히 말하면 프로파일링은 애플리케이션에서 가장 자원이 집중되는 지점을 정밀하게 찾아내는 기법이다. 즉 코드의 병목(bottleneck)을 찾아내고 성능을 측정해주는 도구이다.   
`profile` 모듈, `cProfile` 모듈, `line_profiler` 패키지를 이용하면 높은 수준의 프로파일링이 가능하다. [파이썬 프로파일러 - cProfile, profile](https://docs.python.org/3/library/profile.html), [line profiler를 사용하여 파이썬의 각 라인이 어떻게 돌아가는지를 알아보자.](https://frhyme.github.io/python-libs/python_line_profileing_in_python/)

# 30-4. 멀티태스킹(3) Scale Up vs Scale Out

컴퓨터의 자원을 활용하기 위해 자원을 Up(업그레이드, 최적화) 시킬 수 있고 혹은 자원을 Out(확장) 시킬 수도 있다. __scale-up은 한 대의 컴퓨터 성능을 최적화__시키는 방법이고, __scale-out은 여러 대의 컴퓨터를 한 대처럼 사용__하는 것이다.

# 30-5. 파이썬에서 멀티스레드 사용하기 (1) 스레드 생성

파이썬에서 멀티스레드를 구현하기 위해 `threading` 모듈을 이용하자. [threading - Thread-based parallelism - Python 3.7.12 documentation](https://docs.python.org/3.7/library/threading.html)   

배달과 그릇 찾기 작업을 하는 코드를 통해 알아보자.

In [3]:
class Delivery:
	def run(self):
		print("delivery")

class RetriveDish:
	def run(self):
		print("Retriving Dish")

work1 = Delivery()
work2 = RetriveDish()

def main():
	work1.run()
	work2.run()

if __name__ == '__main__':
    main()

delivery
Retriving Dish


위의 코드를 __멀티스레드__ 방식으로 바꾸면 다음과 같다.   
threading 모듈을 import하고, 클래스에서 thread를 상속했다.

In [4]:
from threading import *

class Delivery(Thread):
	def run(self):
		print("delivery")

class RetriveDish(Thread):
	def run(self):
		print("Retriving Dish")

work1 = Delivery()
work2 = RetriveDish()

def main():
	work1.run()
	work2.run()

if __name__ == '__main__':
    main()

delivery
Retriving Dish


In [5]:
from threading import *

class Delivery:
    def run(self):
        print("delivering")

work1 = Delivery()
print(work1.run)

class Delivery(Thread):
    def run(self):
        print("delivering")

work2 = Delivery()
print(work2.run)

<bound method Delivery.run of <__main__.Delivery object at 0x7ff1bc73e580>>
<bound method Delivery.run of <Delivery(Thread-10, initial)>>


# 30-6. 파이썬에서 멀티스레드 사용하기 (2) 스레드 생성 및 사용

__스레드 생성__   
`threading` 모듈의 `Thread` 클래스를 상속받아 구현할 수 있지만 그대로 인스턴스화해서 스레드를 생성할 수도 있다. `Thread` 클래스에 인자로 `target`, `args` 값을 넣어주면 된다. `args`에 넣은 파라미터는 스레드 함수의 인자로 넘어간다.   

`t = Thread(target=함수이름, args=())`   

Thread 클래스는 `start()`, `join()` 같은 스레드 동작 관련 메소드가 있다. Thread로 실행할 함수를 정의하고 `start()`로 스레드를 실행하게된다. 스레드 함수가 루프 돌 때 꼭 멈춰야 할지 체크하는 flag(여기서는 stopped)를 체크하도록 설계해야한다.

In [6]:
from threading import *
from time import sleep

Stopped = False

def worker(work, sleep_sec):    # 일꾼 스레드입니다.
    while not Stopped:          # 그만 하라고 할때까지
        print('do ', work)      # 시키는 일을 하고
        sleep(sleep_sec)        # 잠깐 쉽니다.
    print('retired..')          # 언젠가 이 굴레를 벗어나면, 은퇴할 때가 오겠지요?
        
t = Thread(target=worker, args=('Overwork', 3))    # 일꾼 스레드를 하나 생성합니다. 열심히 일하고 3초간 쉽니다.
t.start()    # 일꾼, 이제 일을 해야지? 😈

do  Overwork


In [7]:
# 이 코드 블럭을 실행하기 전까지는 일꾼 스레드는 종료하지 않습니다. 
Stopped = True    # 일꾼 일 그만하라고 세팅해 줍시다. 
t.join()          # 일꾼 스레드가 종료할때까지 기다립니다. 
print('worker is gone.')

retired..
worker is gone.


# 30-7. 파이썬에서 멀티프로세스 사용하기

파이썬에서 멀티프로세스 구현은 `multiprocessing` 모듈을 이용하면 된다.   
[multiprocessing - Process-based parallelism - Python 3.7.12 documentation](https://docs.python.org/3.7/library/multiprocessing.html) 

__프로세스 생성__   
스레드와 유사한 방법으로 생성한다. `Process` 인스턴스를 만들고, `target`, `args` 파라미터에 함수 이름과 함수 인자를 전달하면 된다.

In [8]:
import multiprocessing as mp

def delivery():
    print('delivering...')

p = mp.Process(target=delivery, args=())
p.start()

delivering...


__프로세스 사용__   
`Process` 클래스는 `start()`, `join()`, `terminate()` 같은 프로세스 동작 관련 메소드가 있다.   

```
p = mp.Process(target=delivery, args=())
p.start() # 프로세스 시작
p.join() # 실제 종료까지 기다림 (필요시에만 사용)
p.terminate() # 프로세스 종료
```

# 30-8. 파이썬에서 스레드/ 프로세스 풀 사용하기

멀티스레드, 프로세스 작업을 할 때 가장 많은 연산이 필요한 작업은 스레드나 프로세스를 생성하고 종료하는 일이다. 제대로 종료해주어야 컴퓨팅 리소스가 낭비되지 않는다.   
위에서 선언한거처럼 하나씩 실행한다고 전체적인 프로그램의 성능이 좋아지지도 않고 번거로워진다. 그래서 실제로 사용할 때에는 스레드, 프로세스 풀을 사용해서 생성한다.   

풀(Pool)은 스레드나 프로세스들로 가득 찬 풀장이라고 생각하면 된다. 스레드 풀을 만들면 각각 task들에 자동으로 스레드를 할당하고 종료한다. 풀을 만드는 방법은 크게 2가지가 있다. [Queue](https://docs.python.org/3.7/library/queue.html)를 사용해서 만드는 방법과, [concurrent.futures](https://docs.python.org/ko/3.7/library/concurrent.futures.html) 라이브러리의 `ThreadPoolExecutor`, `ProcessPoolExecutor` 클래스를 이용하는 방법이 있다.   
이번에는 `concurrent.futures` 모듈을 사용해서 구현해보자.

`concurrent.futures` 모듈은 크게 4가지의 기능을 갖고 있다. `Executor`, `ThreadPoolExecutor`, `ProcessPoolExecutor`, `Future` 객체 여기서 Thread/Process Pool 구현에 필요한 부분을 살펴보자.   

__`ThreadPoolExecutor`__   
`Executor` 객체를 이용하면 스레드 생성, 시작, 조인 같은 작업할 때 `with` 컨첵스트 관리자와 같은 방법으로 가독성 높은 코드를 구현할 수 있다.   

```
with ThreadPoolExecutor() as executor:
    future = executor.submit(함수이름, 인자)
```

In [9]:
from concurrent.futures import ThreadPoolExecutor

class Delivery:
    def run(self):
        print("delivering")
w = Delivery()

with ThreadPoolExecutor() as executor:
    future = executor.submit(w.run)

delivering


__`multiprocessing.Pool`__   
multiprocessing.Pool.map을 통해 여러 개의 프로세스에 특정 함수를 매핑해서 병렬처리하도록 구현하는 방법이 많이 사용된다.

In [10]:
from multiprocessing import Pool
from os import getpid

def double(i):
    print("I'm processing ", getpid())    # pool 안에서 이 메소드가 실행될 때 pid를 확인해 봅시다.
    return i * 2

with Pool() as pool:
      result = pool.map(double, [1, 2, 3, 4, 5])
      print(result)

I'm processing I'm processing I'm processing  I'm processing  170
 I'm processing 168  
170169
171

[2, 4, 6, 8, 10]


위 `double(i)` 메소드가 pool을 통해 각각 다른 pid를 가진 프로세스들 위에서 multiprocess로 실행된걸 확인할 수 있다.

# 30-9. 실전 예제(1) futures 모듈

`concurrent.futures` 모듈의 `ProcessPoolExecutor`를 이용해 병렬 프로그래밍을 연습해보자.   
[concurrent.futures 모듈 docs](https://docs.python.org/ko/3.7/library/concurrent.futures.html)   

docs를 살펴보면 이 모듈은 `Executor`, `Futures` 객체를 갖고 있고, `Executor`는 `ThreadPoolExecutor`, `ProcessPoolExecutor` 2개의 서브 클래스가 있다. `Executor` 객체가 제공하는 메소드 3개는 `submit()`, `map()`, `shutdown()`가 있음을 확인할 수 있다.

# 30-10. 실전 예제(2) 튜토리얼

이번 예제에서 중점으로 다루는 부분은 2가지가 있다. 
- `Executor` 객체의 `map()` 함수
- `ProcessPoolExecutor` 부분

`concurrent.futures` 모듈의 `ProcessPoolExecutor`를 이용해서 멀티프로세스 구현을 연습해보자. 

In [11]:
import math
import concurrent

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

if __name__ == '__main__':
    main()

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False


위의 코드를 하나씩 뜯어보자.

In [12]:
import math
import concurrent

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

print("*    .\n·   *\n  *   *\n🌲 🦕 🌳")

*    .
·   *
  *   *
🌲 🦕 🌳


소수(prime) 판별 문제로 PRIMES 변수에 선언된 숫자들이 소수인지 아닌지를 판별한다.

In [13]:
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True
print("🌲      🦕...")

🌲      🦕...


소수를 판별하는 로직함수이다. if문으로 0, 1, 2와 짝수를 처리하고 나머지는 math 라이브러리를 활용해서 소수 판별을 진행한다.

In [14]:
def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))
print("🌲   🦕.....")

🌲   🦕.....


`main()` 함수를 작성해서 소수 판별 함수 `is_prime()`을 호출한다.   

맵-리듀스(map-reduce) 스타일로 코드를 작성하고 `map()` 함수를 `ProcessPoolExecutor()` 인스턴스에서 생성된 `executor`에서 실행시킨다. `concurrent.futures` 라이브러리의 프로세스 풀에서 동작하게 하기 위해 `with`문을 써서 구현했다.

병렬과 단일처리의 비교를 위해 코드를 수정해보자.   
- 단일처리 코드 추가. 병렬, 단일처리 프로파일링을 위한 시간 계산 코드 추가.

In [15]:
import time

def main():
    print("병렬처리 시작")
    start = time.time()
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))
    end = time.time()
    print("병렬처리 수행 시각", end-start, 's')
    
    print("단일처리 시작")
    start = time.time()
    for number, prime in zip(PRIMES, map(is_prime, PRIMES)):
        print('%d is prime: %s' % (number, prime))
    end = time.time()
    print("단일처리 수행 시각", end-start, 's')
print(" ❣\n🌲🦕.......")

 ❣
🌲🦕.......


In [16]:
main()

병렬처리 시작
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
병렬처리 수행 시각 2.0041096210479736 s
단일처리 시작
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
단일처리 수행 시각 2.7845189571380615 s
